In [1]:
import cv2
from ultralytics import YOLO
import RPi.GPIO as GPIO
import time

ModuleNotFoundError: No module named 'RPi.GPIO'

In [2]:
# Load the YOLO model
model = YOLO('facemask_detect.pt')

# Define the colors for each label
colors = {
    "Head": (255, 0, 0),       # Blue
    "Nose": (0, 255, 0),       # Green
    "Mouth": (0, 0, 255),      # Red
    "Face Mask": (255, 255, 0) # Cyan
}

# Define the labels
labels = ["Face Mask", "Head", "Mouth", "Nose"]

# GPIO setup for the magnetic lock and buzzer
LOCK_PIN = 18  # Use any available GPIO pin for the lock
BUZZER_PIN = 23  # Use any available GPIO pin for the buzzer

GPIO.setmode(GPIO.BCM)
GPIO.setup(LOCK_PIN, GPIO.OUT)
GPIO.setup(BUZZER_PIN, GPIO.OUT)

GPIO.output(LOCK_PIN, GPIO.LOW)  # Initially, the lock is disengaged

# Initialize PWM for the buzzer
buzzer = GPIO.PWM(BUZZER_PIN, 1000)  # Set frequency to 1 kHz
buzzer.start(0)  # Start PWM with 0% duty cycle (off)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Frame counter for condition
frame_counter = 0
required_frames = 64

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Perform inference
    results = model(frame)

    # Flags for detection
    head_detected = False
    face_mask_detected = False
    nose_detected = False
    mouth_detected = False

    # Parse results
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = labels[int(box.cls[0])]
            color = colors[label]
            conf = box.conf[0]

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            # Display label and confidence
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # Update detection flags
            if label == "Head":
                head_detected = True
            elif label == "Face Mask":
                face_mask_detected = True
            elif label == "Nose":
                nose_detected = True
            elif label == "Mouth":
                mouth_detected = True

    # Check detection conditions
    if head_detected and face_mask_detected and not nose_detected and not mouth_detected:
        frame_counter += 1
    else:
        frame_counter = 0

    # Control the magnetic lock and buzzer
    if frame_counter >= required_frames:
        GPIO.output(LOCK_PIN, GPIO.HIGH)  # Engage the lock
        buzzer.ChangeDutyCycle(50)  # 50% duty cycle (beep on)
        time.sleep(0.5)  # Beep for 0.5 second
        buzzer.ChangeDutyCycle(0)  # 0% duty cycle (beep off)
    else:
        GPIO.output(LOCK_PIN, GPIO.LOW)   # Disengage the lock

    # Display the resulting frame
    cv2.imshow('Frame', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()

# Cleanup GPIO
GPIO.cleanup()


SyntaxError: invalid syntax (1740611921.py, line 15)